### Docker
### docker pull qdrant/qdrant
### docker run -p 6333:6333 -p 6334:6334 \
### -v "$(pwd)/qdrant_storage:/qdrant/storage:z" qdrant/qdrant

## Qdrant is now accessible:
- REST API: localhost:6333
- Web UI: localhost:6333/dashboard
- GRPC API: localhost:6334

## Initialize the client

In [2]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

## Create a collection

In [3]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

True

## Add vectors

In [4]:
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)

operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>


## Run a query

In [5]:
search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=False,
    limit=3
).points

print(search_result)

[ScoredPoint(id=4, version=1, score=1.362, payload=None, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=1, score=1.273, payload=None, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=1, score=1.208, payload=None, vector=None, shard_key=None, order_value=None)]


## Add a filter

In [6]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    with_payload=True,
    limit=3,
).points

print(search_result)

[ScoredPoint(id=2, version=1, score=0.871, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None)]
